In [8]:
from nltk.corpus import movie_reviews
import nltk

In [24]:
import numpy as np
import pandas as pd

In [9]:
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Mi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


True

Получили id-шники негативных и позитивных отзывов:

In [49]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

Получили список отзывов:

In [50]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

In [52]:
list_words = np.array(negfeats + posfeats)
data = pd.DataFrame(list_words)
data.columns = ['texts']

In [54]:
data['label'] = [0]*len(negids) + [1]*len(posids)

In [55]:
data.head()

,texts,label
0,"(plot, :, two, teen, couples, go, to, a, churc...",0
1,"(the, happy, bastard, ', s, quick, movie, revi...",0
2,"(it, is, movies, like, these, that, make, a, j...",0
3,"("", quest, for, camelot, "", is, warner, bros, ...",0
4,"(synopsis, :, a, mentally, unstable, man, unde...",0


Количество отзывов во всей выборке

In [56]:
len(data)

2000

Доля позитивных отзывов в выборке

In [57]:
len(posids)/len(data)

0.5

In [58]:
from sklearn.feature_extraction.text import CountVectorizer 

In [76]:
texts = negfeats + posfeats
new_texts = []
for i in range(len(texts)):
    new_texts.append(str(' '.join(texts[i])))

In [78]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(np.array(new_texts))

Количество признаков в CountVectorizer: 39659

Далее собрали pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оценили получаемое "из коробки" качество по accuracy.

In [81]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [83]:
my_pipeline= Pipeline([('count_vec', CountVectorizer()), 
                      ('reg', LogisticRegression())])

In [85]:
accuracy_mean = cross_val_score(my_pipeline, new_texts, data['label'], scoring='accuracy', cv=4).mean()

C:\Users\Mi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [86]:
print('Средняя точность по кросс-валидации = ', accuracy_mean)

Средняя точность по кросс-валидации =  0.838


Аналогично accuracy, оценили качество по ROC AUC.

In [87]:
roc_mean = cross_val_score(my_pipeline, new_texts, data['label'], scoring='roc_auc', cv=4).mean()

C:\Users\Mi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Mi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [88]:
print('Средняя точность по кросс-валидации = ', roc_mean)

Средняя точность по кросс-валидации =  0.914032


Обучили логистическую регрессию на всей доступной выборке и определили 5 наиболее важных для модели признаков.

In [97]:
Features = vectorizer.get_feature_names()

log_reg = LogisticRegression()
log_reg.fit(X, data['label'])

C:\Users\Mi\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [126]:
weights = log_reg.coef_
abs_weights = np.abs(weights)
Imp_indexes = abs_weights[0].argsort()[-5:]

наиболее важные признаки

In [127]:
np.array(Features)[Imp_indexes]

array(['waste', 'fun', 'worst', 'unfortunately', 'bad'], dtype='<U58')